In [1]:
import pandas as pd
import numpy as np
import pathlib
from time import time


In [2]:
import warnings
warnings.filterwarnings("ignore")


In [3]:
ptr_df_columns = ['TEST_NUM', 'SITE_NUM', 'TEST_FLG', 'RESULT', 
                  'LO_LIMIT', 'HI_LIMIT', 'UNITS']
ftr_df_columns = ['TEST_NUM', 'SITE_NUM', 'TEST_FLG']
prr_df_columns = ['PART_ID', 'PART_FLG', 'NUM_TEST']
config_df_columns = ['TEST_NUM',"T_Type","LO_LIMIT",'HI_LIMIT', 'UNITS']
final_df_columns = ['TEST_NUM', 'PART_ID', 'Status',"LO_LIMIT",
                    'HI_LIMIT', 'UNITS', 'P_Type']

non_exe_columns = ["TEST_NUM", "SITE_NUM","TEST_FLG","RESULT","LO_LIMIT", 
                   "HI_LIMIT","UNITS","P_Type","PART_ID","STATUS"]


In [4]:

def getDataset(masterfile,config_file):
    # Reading all files
    ptr_df = pd.read_excel(masterfile, sheet_name = 'PTR')[ptr_df_columns]
    ftr_df = pd.read_excel(masterfile, sheet_name = 'FTR')[ftr_df_columns]
    prr_df = pd.read_excel(masterfile, sheet_name = 'PRR')[prr_df_columns]

    config_df = pd.read_csv(config_file)
    
    # empty data frame template
    df_final = pd.DataFrame(columns=final_df_columns)
    non_exe_df = pd.DataFrame(columns=non_exe_columns)
    
    return ptr_df, ftr_df, prr_df, config_df, df_final, non_exe_df


In [5]:

def getExpandReport(df_final, prr_df, config_df):
    # Expand result and will bring PART_ID in final data 
    for row in prr_df.itertuples():
        # PART_FLG == 0 denotes those record is pass in PRR data
        if row.PART_FLG == 0:         
            part_1 = config_df[['TEST_NUM']]
            part_1['PART_ID'] = row.PART_ID
            part_1['Status'] = 'Pass'
            part_1['LO_LIMIT'] = config_df['LO_LIMIT']
            part_1['HI_LIMIT'] = config_df['HI_LIMIT']
            part_1['UNITS'] = config_df['UNITS']
            df_final = df_final.append(part_1)

        else:        
            part_1 = config_df[['TEST_NUM']]
            part_1['PART_ID'] = row.PART_ID 
            part_1['Status'] = 'Fail' 
            part_1['LO_LIMIT'] = config_df['LO_LIMIT']
            part_1['HI_LIMIT'] = config_df['HI_LIMIT']
            part_1['UNITS'] = config_df['UNITS']
            part_1.iloc[0:row.NUM_TEST,[2]] = 'Pass'
            df_final = df_final.append(part_1)
    return df_final

In [6]:

def updatePartId(prr_df, ftr_df, ptr_df):
    ftr_counter = 0
    ptr_counter = 0
    ftr_df['PART_ID'] = 0
    ptr_df['PART_ID'] = 0

    # Updating PATR_ID in both FTR and PTR datas
    for row in prr_df.itertuples():
        ftr_df.iloc[ftr_counter:ftr_counter+int(row.FTR),-1] = row.PART_ID
        ftr_counter += int(row.FTR)
        if not pd.isnull(row.PTR):
            ptr_df.iloc[ptr_counter:ptr_counter+int(row.PTR),-1] = row.PART_ID
            ptr_counter += int(row.PTR)
    return ftr_df, ptr_df

In [7]:

def fillMissedData(df,config_df):
    # Filling LO_LIMIT, HI_LIMIT, UNITS NaN with existing Configure File
    df.LO_LIMIT = df.LO_LIMIT.mask(df.LO_LIMIT.isna()).fillna(    
    df.TEST_NUM.map(config_df.set_index('TEST_NUM').LO_LIMIT))
    
    df.HI_LIMIT = df.HI_LIMIT.mask(df.HI_LIMIT.isna()).fillna(    
    df.TEST_NUM.map(config_df.set_index('TEST_NUM').HI_LIMIT))   
    
    df.UNITS = df.UNITS.mask(df.UNITS.isna()).fillna(    
    df.TEST_NUM.map(config_df.set_index('TEST_NUM').UNITS))  
    
    return df

In [8]:

def updateResult(df):
    # Updating Fail Pass result for FTR 
    df.loc[(df['P_Type']=="FTR")&(df['RESULT']==1),"STATUS"] = 'Pass'
    df.loc[(df['P_Type']=="FTR")&(df['RESULT']!=1),"STATUS"] = 'Fail'
    
    # Updating Fail Pass result for PTR 
    df.loc[(df['P_Type']=="PTR")&((df['RESULT']>=df.LO_LIMIT)&
           (df['RESULT']<=df.HI_LIMIT)),"STATUS"] = 'Pass'
    df.loc[(df['P_Type']=="PTR")&((df['RESULT']<df.LO_LIMIT)|
           (df['RESULT']>df.HI_LIMIT)),"STATUS"] = 'Fail'
    
    return df


# In[11]:


def getNonExeDf(df,non_exe_df):
    df = df.query('Status == "Fail"')
    non_exe_df['TEST_NUM'] = df['TEST_NUM']
    non_exe_df['PART_ID'] = df['PART_ID']
    non_exe_df['STATUS'] = 'Non_Executed'
    non_exe_df['P_Type'] = df['Record_type']
    return non_exe_df

In [9]:

def prepareData(masterfile,config_file,float_precision='round_trip'):
    # get All datas and data templates
    ptr_df, ftr_df, prr_df, config_df, df_final, non_exe_df = getDataset(masterfile,config_file)

    # initilize colums
    ftr_df[['RESULT', 'LO_LIMIT', 'HI_LIMIT', 'UNITS']] = np.NaN
    
    # Assuming all result values is 0 initially it denotes FAIL
    ftr_df['RESULT'] = 0
    
    # Adding types of test in P_Type column 
    ptr_df['P_Type'] = 'PTR'
    ftr_df['P_Type'] = 'FTR'

    # Creating Result column and mark it as 1 where test_flg is 0
    ftr_df.loc[ftr_df['TEST_FLG'] == 0, 'RESULT'] = 1
    
    # Expanding Report for all part ids
    df_final = getExpandReport(df_final, prr_df, config_df)
    df_final.to_csv("df_final.csv",index=False)
    
    # print(f'df_final {df_final.shape}')
    # update Record_type PTR or FTR
    op_df = df_final
    op_df["Record_type"] = op_df["TEST_NUM"].map(config_df[['TEST_NUM','T_Type']]                                             .set_index('TEST_NUM')['T_Type'],)
    op_df.to_excel("Record_types.xlsx",index=False)
    print(f'op_df {op_df.shape}')
    
    # updating FTR and PTR counts in PRR data
    prr_df[["FTR","PTR"]] = op_df.loc[op_df['Status']=="Pass"]                .groupby('PART_ID')['Record_type'].value_counts()                .unstack()[["FTR","PTR"]].values
    prr_df.to_excel("Mapped_PRR.xlsx",index=False)
    
    ftr_df, ptr_df = updatePartId(prr_df, ftr_df, ptr_df)
    
    # Merging FTR and PTR together & sorting based on TEST_NUM
    complete_df = ftr_df.append(ptr_df)
    output_df = complete_df.groupby('PART_ID').apply(lambda x: x.sort_values('TEST_NUM'))
    output_df.to_excel("Final_output.xlsx",index=False)
    
    print(f'output_df {output_df.shape}')
    
    df = fillMissedData(output_df,config_df)
    df = updateResult(df)
    # df = updateResult(output_df)
    print(f'df {df.shape}')
    
    # creating Non executed Records
    non_exe_df = getNonExeDf(op_df,non_exe_df)
    non_exe_df = fillMissedData(non_exe_df,config_df)
    
    df = df.append(non_exe_df)
    print(f'non_exe_df {non_exe_df.shape}')
    
    df.to_csv("CheckingforFTR.csv",index=False,float_format="%.5f")
    

In [10]:
if __name__=="__main__":
    masterfile = 'aa.xlsx'
    config_file = 'TEST_NUM.csv'
    prepareData(masterfile,config_file)


op_df (41328, 8)
output_df (36254, 9)
df (36254, 10)
non_exe_df (5074, 10)


In [13]:
# convT_df = pd.read_csv("CheckingforFTR - Copy.csv")
# data=convT_df.pivot(index='PART_ID',columns='TEST_NUM',values='STATUS')
# #df.to_csv("Converted_CheckingforFTR.csv"#,index=False,float_format="%.5f")


In [14]:
# data.to_csv("converteddata_CheckingforFTR.csv",index=False,float_format="%.5f")

# Not

In [15]:
# import pandas as pd
# #dataR=pd.read_excel("Final_output_Copy.xlsx")
# dataR=pd.read_csv("CheckingforFTR - Copy.csv")

# #print(dataR)
# data1_tra=data.T
# #print(data1_tra)
# dataR

TEST_NUM  SITE_NUM  TEST_FLG    RESULT  LO_LIMIT  HI_LIMIT UNITS  \
0           100       0.0       0.0  1.000000       NaN       NaN   NaN   
1           110       0.0       0.0  1.000000       NaN       NaN   NaN   
2           200       0.0       0.0  2.000000     1.000     3.000     v   
3           210       0.0       0.0  0.043458     0.035     0.055     a   
4           220       0.0       0.0  0.046440     0.035     0.060     a   
...         ...       ...       ...       ...       ...       ...   ...   
41323       743       NaN       NaN       NaN     0.000     0.000     s   
41324       744       NaN       NaN       NaN     0.000     0.000     s   
41325       745       NaN       NaN       NaN     0.000     0.000     s   
41326       746       NaN       NaN       NaN     0.000     0.000     s   
41327       747       NaN       NaN       NaN     0.000     0.000     s   

      P_Type  PART_ID        STATUS  
0        FTR        1          Pass  
1        FTR        1          Pass  
2        PTR        1          Pass  
3        PTR        1          Pass  
4        PTR        1          Pass  
...      ...      ...           ...  
41323    PTR      336  Non_Executed  
41324    PTR      336  Non_Executed  
41325    PTR      336  Non_Executed  
41326    PTR      336  Non_Executed  
41327    PTR      336  Non_Executed  

[41328 rows x 10 columns]

In [16]:
# #data_tra.to_excel("Final_output_converted.xlsx")
# data1_tra.to_csv("Formet2.csv")